In [ ]:
!pip install imutils 
!pip install gdown

In [ ]:
!git clone https://github.com/shilu10/MIRNet-TF2.git

In [ ]:
!mv MIRNet-TF2 MIRNet_W1

In [ ]:
!cp /kaggle/working/MIRNet_TF0/checkpoint/saved/enchancement/best_model.h5 .

In [ ]:
!mv MIRNet-TF2 MIRNet_TF0

In [ ]:
import tensorflow as tf 

model = tf.keras.models.load_model("best_model.h5")

In [ ]:
!mv MIRNet_TF8/SIDD_Small_sRGB_Only MIRNet_TF9

In [ ]:
!python train_enhancement.py --loss_function=charbonnier --n_epochs=200 --batch_size=10 --num_rrg=2 --num_mrb=2

In [ ]:
import cv2             

In [ ]:
cv2.imread('GT_SRGB_010.PNG').shape

In [ ]:
!pip install imutils 
!pip install gdown

In [ ]:
import gdown 
import os 
import shutil 
from imutils import paths 
import glob 
import glob 
import numpy as np 
from tensorflow import keras 
import tensorflow as tf 
from tensorflow.keras import *


from tensorflow import keras 
import tensorflow as tf 
import numpy as np 


def random_crop(lr_img, hr_img, hr_crop_size=128):
    lr_crop_size = hr_crop_size
    lr_img_shape = tf.shape(lr_img)[:2]

    lr_w = tf.random.uniform(shape=(), maxval=lr_img_shape[1] - lr_crop_size + 1, dtype=tf.int32)
    lr_h = tf.random.uniform(shape=(), maxval=lr_img_shape[0] - lr_crop_size + 1, dtype=tf.int32)

    hr_w = lr_w
    hr_h = lr_h

    lr_img_cropped = lr_img[lr_h:lr_h + lr_crop_size, lr_w:lr_w + lr_crop_size]
    hr_img_cropped = hr_img[hr_h:hr_h + hr_crop_size, hr_w:hr_w + hr_crop_size]

    return lr_img_cropped, hr_img_cropped


def random_flip(lr_img, hr_img):
    rn = tf.random.uniform(shape=(), maxval=1)
    return tf.cond(rn < 0.5,
                   lambda: (lr_img, hr_img),
                   lambda: (tf.image.flip_left_right(lr_img),
                            tf.image.flip_left_right(hr_img)))


def random_rotate(lr_img, hr_img):
    rn = tf.random.uniform(shape=(), maxval=4, dtype=tf.int32)
    return tf.image.rot90(lr_img, rn), tf.image.rot90(hr_img, rn)


def random_crop_sr(lr_img, hr_img, hr_crop_size=96, scale=2):
    lr_crop_size = hr_crop_size // scale
    lr_img_shape = tf.shape(lr_img)[:2]

    lr_w = tf.random.uniform(shape=(), maxval=lr_img_shape[1] - lr_crop_size + 1, dtype=tf.int32)
    lr_h = tf.random.uniform(shape=(), maxval=lr_img_shape[0] - lr_crop_size + 1, dtype=tf.int32)

    hr_w = lr_w * scale
    hr_h = lr_h * scale

    lr_img_cropped = lr_img[lr_h:lr_h + lr_crop_size, lr_w:lr_w + lr_crop_size]
    hr_img_cropped = hr_img[hr_h:hr_h + hr_crop_size, hr_w:hr_w + hr_crop_size]

    return lr_img_cropped, hr_img_cropped


class UnsuuportedFileExtension(Exception):
    def __init__(self, message):
        self.message = message

        
class InitializationErro(Exception):
    def __init__(self, message):
        self.message = message
        

import gdown 
import os 
import shutil 
from imutils import paths 
import glob 
import glob 
import numpy as np 
from tensorflow import keras 
import tensorflow as tf 
from tensorflow.keras import *



class UnsuuportedFileExtension(Exception):
    def __init__(self, message):
        self.message = message

        
class InitializationErro(Exception):
    def __init__(self, message):
        self.message = message
        

class SIDDDataLoader:
    def __init__(self, dname):
        assert dname in ["sidd"], "given dataset name is not valid, supported datasets are ['lol']"  
        #assert type(resize_shape) == int, 'Unknown dtype for resize shape, needed Int' 
        #assert type(batch_size) == int, 'Unknown dtype for batch_size, needed Int' 
        self.dname = dname 
    
    def __image_files(self):
        try:
            with open('SIDD_Small_sRGB_Only/Scene_Instances.txt') as f:
                instances = f.read()

            instances = instances.split('\n')
            path = 'SIDD_Small_sRGB_Only/Data/'

            noisy_images_path = []
            gt_images_path = []

            for f in instances:
                images_path = path + f + '/'

                for g in os.listdir(images_path):
                    image_path = images_path + g

                    if 'NOISY' in image_path:
                        noisy_images_path.append(image_path)
                    else:
                        gt_images_path.append(image_path)

            return noisy_images_path, gt_images_path
        
        except Exception as err:
            return err
            
    def __noisy_image_files(self):
        try:
            num_train = 150
            
            noisy_images_path, _ = self.__image_files()
            train_noisy_data_path = noisy_images_path[: num_train]
            val_noisy_data_path = noisy_images_path[num_train+1: ]

            return train_noisy_data_path, val_noisy_data_path
        
        except Exception as err:
            return err
    
    def __gt_image_files(self):
        try:
            num_train = 150
            
            _, gt_images_path = self.__image_files()
            train_gt_data_path = gt_images_path[: num_train]
            val_gt_data_path = gt_images_path[num_train+1: ]

            return train_gt_data_path, val_gt_data_path
        
        except Exception as err:
            return err
    
    def __train_tf_dataset(self):
        try: 
            noisy_train_files, _ = self.__noisy_image_files()
            gt_train_files, _ = self.__gt_image_files()
            
            tf_dataset = tf.data.Dataset.from_tensor_slices((noisy_train_files, gt_train_files)) 
            return tf_dataset
        
        except Exception as err:
            return err 
    
    def __val_tf_dataset(self):
        try: 
            _, noisy_val_files = self.__noisy_image_files()
            _, gt_val_files = self.__gt_image_files()
            tf_dataset = tf.data.Dataset.from_tensor_slices((noisy_val_files, gt_val_files)) 
            return tf_dataset 
        
        except Exception as err:
            return err
    
    def initialize(self):
        try: 
            if self.dname == "sidd":
                SIDD_DATA_PATH = 'https://competitions.codalab.org/my/datasets/download/a26784fe-cf33-48c2-b61f-94b299dbc0f2'

                if not os.path.exists('SIDD_Small_sRGB_Only/Data'):
                    #os.system(f'wget {SIDD_DATA_PATH}')
                    # https://drive.google.com/file/d/10TC19ND0qeqUCG_mvfhtOjmXkSunYyud/view?usp=sharing
                    url = "https://drive.google.com/uc?id=10TC19ND0qeqUCG_mvfhtOjmXkSunYyud"
                    gdown.download(url)
                    os.system(f'unzip -q smartphone-image-denoising-dataset.zip')
                    os.system(f'rm smartphone-image-denoising-dataset.zip')

                if (os.path.exists("smartphone-image-denoising-dataset.zip") and not os.path.exists("SIDD_Small_sRGB_Only/")):
                    os.system(f'unzip -q smartphone-image-denoising-dataset.zip')
                    os.system(f'rm smartphone-image-denoising-dataset.zip')
                    
        except Exception as err:
            print(err, "rr")
            return err 
        
    def __read_img(self, img_fpath): 
        try: 
            raw = tf.io.read_file(img_fpath)
            image = tf.image.decode_png(raw, channels=3)
            image = tf.image.convert_image_dtype(image, dtype=tf.float32)
            return image
            
        except Exception as err:
            return err
        
    def __load_data(self, lr_img_path, hr_img_path):
        try: 
            lr_img = self.__read_img(lr_img_path)
            hr_img = self.__read_img(hr_img_path)

            # resizing
           # lr_img = tf.image.resize(lr_img, (224, 224))
           # hr_img = tf.image.resize(hr_img, (224, 224))

            return lr_img, hr_img
        
        except Exception as err:
            return err
    
    def image_resize(self, lr_img, hr_img):
        # resizing
        lr_img = tf.image.resize(lr_img, (128, 128))
        hr_img = tf.image.resize(hr_img, (128, 128))
        
        return lr_img, hr_img

    def __create_tf_dataset(self, tf_ds, batch_size, transform):
        
        if transform:
            tf_ds = tf_ds.map(lambda lr, hr: random_crop(lr, hr), num_parallel_calls=tf.data.AUTOTUNE)
            tf_ds = tf_ds.map(random_flip, num_parallel_calls=tf.data.AUTOTUNE)
            tf_ds = tf_ds.map(random_rotate, num_parallel_calls=tf.data.AUTOTUNE)
            tf_ds = tf_ds.batch(batch_size, drop_remainder=False)

        if not transform:
            tf_ds = tf_ds.map(lambda lr, hr: self.image_resize(lr, hr), num_parallel_calls=tf.data.AUTOTUNE)
            tf_ds = tf_ds.batch(1, drop_remainder=False)
        
        tf_ds = tf_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
        
        return tf_ds
        
    def get_dataset(self, subset, batch_size, transform=True):
        assert subset in ("train", 'val'), "unsupported split type"
        try:
            if subset == "train":
                tf_ds = self.__train_tf_dataset()
                tf_ds = tf_ds.map(self.__load_data, num_parallel_calls=tf.data.AUTOTUNE)
                tf_ds = self.__create_tf_dataset(tf_ds, batch_size, transform)
                return tf_ds
            
            else:
                tf_ds = self.__val_tf_dataset()
                tf_ds = tf_ds.map(self.__load_data, num_parallel_calls=tf.data.AUTOTUNE)
                tf_ds = self.__create_tf_dataset(tf_ds, batch_size, transform)
                return tf_ds
                
        except Exception as err:
            print(err)
            raise InitializationErro('DataLoader, has not been initialize, use .initalize method')


In [ ]:
sl = SIDDDataLoader('sidd')

In [ ]:
sl.initialize()

In [ ]:
t_ds = sl.get_dataset("train", 5)

In [ ]:
len(t_ds)

In [ ]:
for i in t_ds:
    print(i[0].shape, i[1].shape)
    #break

In [ ]:
v_ds = sl.get_dataset("val", 5)

In [ ]:
v_ds

In [ ]:
for i in v_ds:
    print(i[0].shape)

In [ ]:
!git clone https://github.com/shilu10/MIRNet-TF2.git

In [ ]:
!mv MIRNet-TF2 MIRNet_enhancement

In [ ]:
!echo "from .models import get_enhancement_model, get_denoising_model, get_super_resolution_model" >> __init__.py

In [ ]:
!python train_enhancement.py --loss_function=charbonnier --n_epochs=100 --batch_size=5 --num_rrg=2 --num_mrb=2 --use_custom_trainer=False

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
import tensorflow as tf 

In [ ]:
model_path = "pretrained_weights/super_resolution/best_model.h5"

In [ ]:
from mirnet import get_enhancement_model

In [ ]:
model = get_enhancement_model(
            num_rrg=1,
            num_mrb=1,
            num_channels=64
        )

In [ ]:
model.load_weights(model_path)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [ ]:
tflite_model = converter.convert()

In [ ]:
tflite_model_path = "pretrained_weights/super_resolution/best_model.tflite"

with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [ ]:
tflite_model_path

In [ ]:
def inferrer(image, input_dims=(1, 400, 600, 3)):
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.resize_tensor_input(input_details[0]['index'], input_dims)

    input_index = input_details[0]["index"]
    output_index = output_details[0]["index"]

    interpreter.allocate_tensors()
    interpreter.set_tensor(input_index, image)

    interpreter.invoke()
    output = interpreter.get_tensor(output_index)
    
    # Convert output array to image
    # output_image = (np.squeeze(output, axis=0).clip(0, 1) * 255).astype(np.uint8)

    # img = Image.fromarray(output_image)
    
    return output[0]

In [ ]:
import glob, time

lowlight_test_images_path = "test/enhancement/"
test_files = glob.glob(lowlight_test_images_path + "*.png")

for test_file in tqdm.tqdm(test_files, total=len(test_files)):
    
        filename = test_file.split("/")[-1]
        lr_img = cv2.imread(test_file)
        lr_img = cv2.cvtColor(lr_img, cv2.COLOR_BGR2RGB)
        
        # for resizing specific model data to specific dim.
        lr_img = get_lowres_image(lr_img, mode="enhancement")
        
        inputs = img_to_array(lr_img)
        inputs = np.expand_dims(inputs, axis=0)
        t = time.time()
        
        lr_img_shape = lr_img.shape
        inferrer_input_dims = (1, lr_img_shape[0], lr_img_shape[1], lr_img_shape[2])
        enhanced_image = inferrer(inputs, inferrer_input_dims)
        print("Time taken for inference: ", time.time() - t)

        if False:
            plt.figure()
            plt.subplot(131)
            plt.imshow(original_img)
            
            plt.subplot(132)
            plt.imshow(enhanced_image)
            
            plt.show()
        
        
        save_file_dir = lowlight_test_images_path.replace('test', 'results')
        save_file_path = save_file_dir  + filename
        print(save_file_path, "save path")
        cv2.imwrite(save_file_path, cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB))

In [ ]:
img_paths = glob.glob("results/enhancement/*.png")
for img_path in img_paths:
#    n_rows = len(img_paths)
    img = cv2.imread(img_path)
    
    print(img.shape, "result shape")

    plt.figure()
    plt.imshow(img)
    plt.show()

In [ ]:
4

In [ ]:
import tqdm
import numpy as np 
import tensorflow as tf 
import PIL
from PIL import Image
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt

In [ ]:
def get_lowres_image(img, mode="enhancement", scale_factor=4):
    if mode == "denoise":
        img = cv2.resize(img, (512, 360))
    
    if mode == "super_resolution":
        img = cv2.resize(img, (400, 260))
        
    else:
        img = img 
    
    return img

In [ ]:
def inferrer(image, input_dims=(1, 400, 600, 3)):
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.resize_tensor_input(input_details[0]['index'], input_dims)

    input_index = input_details[0]["index"]
    output_index = output_details[0]["index"]

    interpreter.allocate_tensors()
    interpreter.set_tensor(input_index, image)

    interpreter.invoke()
    output = interpreter.get_tensor(output_index)
    
    # Convert output array to image
    # output_image = (np.squeeze(output, axis=0).clip(0, 1) * 255).astype(np.uint8)

    # img = Image.fromarray(output_image)
    
    return output[0]

In [ ]:
import glob, time

lowlight_test_images_path = "test/enhancement/"
test_files = glob.glob(lowlight_test_images_path + "*.png")

for test_file in tqdm.tqdm(test_files, total=len(test_files)):
    
        filename = test_file.split("/")[-1]
        lr_img = cv2.imread(test_file)
        lr_img = cv2.cvtColor(lr_img, cv2.COLOR_BGR2RGB)
        
        # for resizing specific model data to specific dim.
        lr_img = get_lowres_image(lr_img, mode="enhancement")
        
        inputs = img_to_array(lr_img)
        inputs = np.expand_dims(inputs, axis=0)
        t = time.time()
        
        lr_img_shape = lr_img.shape
        inferrer_input_dims = (1, lr_img_shape[0], lr_img_shape[1], lr_img_shape[2])
        enhanced_image = inferrer(inputs, inferrer_input_dims)
        print("Time taken for inference: ", time.time() - t)

        if False:
            plt.figure()
            plt.subplot(131)
            plt.imshow(original_img)
            
            plt.subplot(132)
            plt.imshow(enhanced_image)
            
            plt.show()
        
        
        save_file_dir = lowlight_test_images_path.replace('test', 'results')
        save_file_path = save_file_dir  + filename
        print(save_file_path, "save path")
        cv2.imwrite(save_file_path, cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB))

In [ ]:
img_paths = glob.glob("results/enhancement/*.png")
for img_path in img_paths:
#    n_rows = len(img_paths)
    img = cv2.imread(img_path)
    
    print(img.shape, "result shape")

    plt.figure()
    plt.imshow(img)
    plt.show()

In [ ]:
!pip install gdown 
!pip install imutils 

In [ ]:
!python train_denoise.py --num_rrg=1 --num_mrb=1 --n_epochs=2 --batch_size=5 --use_custom_trainer=True

In [ ]:
# enhancement 
# denoise 
# super_resolution
# tflite.

In [ ]:
from mirnet import get_denoising_model

In [ ]:
denoise = get_denoising_model(num_rrg=2, num_mrb=2, num_channels=64)

In [ ]:
denoise.load_weights("pretrained_weights/denoise/best_model.h5")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(denoise)
tflite_model = converter.convert()
with open("pretrained_weights/denoise/best_model.tflite", 'wb') as f:
    f.write(tflite_model)

In [ ]:
def inferrer(image, input_dims=(1, 400, 600, 3)):
    interpreter = tf.lite.Interpreter(model_path="pretrained_weights/denoise/best_model.tflite")
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.resize_tensor_input(input_details[0]['index'], input_dims)

    input_index = input_details[0]["index"]
    output_index = output_details[0]["index"]

    interpreter.allocate_tensors()
    interpreter.set_tensor(input_index, image)

    interpreter.invoke()
    output = interpreter.get_tensor(output_index)
    
    # Convert output array to image
    # output_image = (np.squeeze(output, axis=0).clip(0, 1) * 255).astype(np.uint8)

    # img = Image.fromarray(output_image)
    
    return output[0]

In [ ]:
import glob, time

lowlight_test_images_path = "test/denoise/"
test_files = glob.glob(lowlight_test_images_path + "*.PNG")

for test_file in tqdm.tqdm(test_files, total=len(test_files)):
    
        filename = test_file.split("/")[-1]
        lr_img = cv2.imread(test_file)
        lr_img = cv2.cvtColor(lr_img, cv2.COLOR_BGR2RGB)
        print(lr_img.shape)
        
        # for resizing specific model data to specific dim.
        lr_img = get_lowres_image(lr_img, mode="denoise")
        print(lr_img.shape)
        
        inputs = img_to_array(lr_img)
        inputs = np.expand_dims(inputs, axis=0)
        t = time.time()
        
        lr_img_shape = lr_img.shape
        inferrer_input_dims = (1, lr_img_shape[0], lr_img_shape[1], lr_img_shape[2])
        enhanced_image = inferrer(inputs, inferrer_input_dims)
        print("Time taken for inference: ", time.time() - t)

        if False:
            plt.figure()
            plt.subplot(131)
            plt.imshow(original_img)
            
            plt.subplot(132)
            plt.imshow(enhanced_image)
            
            plt.show()
        
        
        save_file_dir = lowlight_test_images_path.replace('test', 'results')
        save_file_path = save_file_dir  + filename
        print(save_file_path, "save path")
        cv2.imwrite(save_file_path, cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB))

In [ ]:
img_paths = glob.glob("results/denoise/*.PNG")
for img_path in img_paths:
#    n_rows = len(img_paths)
    img = cv2.imread(img_path)
    
    print(img.shape, "result shape")

    plt.figure()
    plt.imshow(img)
    plt.show()

In [ ]:
# if tflite converted, 
    # we need to test denoise tflite model.
    # test the code for testin_denoise
    # test the code for super_resolution
    # tflite sr.
    # tssting sr code.

In [ ]:
!python train_super_resolution.py --n_epochs=3 --batch_size=5 --num_rrg=1 --num_mrb=1 --use_custom_trainer=True --loss_function=charbonnier

In [ ]:
!echo "from .models import get_enhancement_model, get_denoising_model, get_super_resolution_model" >> __init__.py

In [ ]:
!mv MIRNet_W/.div2k MIRNet_W1

In [ ]:
!mv /kaggle/working/MIRNet_W/SIDD_Small_sRGB_Only MIRNet_W1

In [ ]:
!cat __init__.py

In [ ]:
from mirnet import get_super_resolution_model

In [ ]:

# Reference: https://github.com/krasserm/super-resolution/blob/master/data.py
import gdown 
import os 
import shutil 
from imutils import paths 
import glob 
from glob import glob 
import numpy as np 
from tensorflow import keras 
import tensorflow as tf 
from tensorflow.keras import *
from tensorflow.python.data.experimental import AUTOTUNE
from dataloaders.utils import random_crop, random_crop_sr, random_flip, random_rotate


class SRDataLoader1:
    def __init__(self,
                 scale=2,
                 subset='train',
                 downgrade='bicubic',
                 images_dir='.div2k/images',
                 caches_dir='.div2k/caches'):

        self._ntire_2018 = True

        _scales = [2, 3, 4, 8]

        if scale in _scales:
            self.scale = scale
        else:
            raise ValueError(f'scale must be in ${_scales}')

        if subset == 'train':
            self.image_ids = range(1, 801)
        elif subset == 'valid':
            self.image_ids = range(801, 901)
        else:
            raise ValueError("subset must be 'train' or 'valid'")

        _downgrades_a = ['bicubic', 'unknown']
        _downgrades_b = ['mild', 'difficult']

        if scale == 8 and downgrade != 'bicubic':
            raise ValueError(f'scale 8 only allowed for bicubic downgrade')

        if downgrade in _downgrades_b and scale != 4:
            raise ValueError(f'{downgrade} downgrade requires scale 4')

        if downgrade == 'bicubic' and scale == 8:
            self.downgrade = 'x8'
        elif downgrade in _downgrades_b:
            self.downgrade = downgrade
        else:
            self.downgrade = downgrade
            self._ntire_2018 = False

        self.subset = subset
        self.images_dir = images_dir
        self.caches_dir = caches_dir

        os.makedirs(images_dir, exist_ok=True)
        os.makedirs(caches_dir, exist_ok=True)

    def __len__(self):
        return len(self.image_ids)

    def dataset(self, batch_size=16, repeat_count=None, random_transform=True):
        ds = tf.data.Dataset.zip((self.lr_dataset(), self.hr_dataset()))
        if random_transform:
            ds = ds.map(lambda lr, hr: random_crop_sr(lr, hr, scale=self.scale), num_parallel_calls=AUTOTUNE)
            ds = ds.map(random_rotate, num_parallel_calls=AUTOTUNE)
            ds = ds.map(random_flip, num_parallel_calls=AUTOTUNE)
            ds = ds.map(lambda lr, hr: (tf.image.convert_image_dtype(lr, tf.float32), tf.image.convert_image_dtype(hr, tf.float32)))
        
        if not random_transform:
            ds = ds.map(lambda lr, hr: random_crop_sr(lr, hr, scale=self.scale), num_parallel_calls=AUTOTUNE)
      #      ds = ds.map(lambda lr, hr: lr/255.0, hr/255.0)
            ds = ds.map(lambda lr, hr: (tf.image.convert_image_dtype(lr, tf.float32), tf.image.convert_image_dtype(hr, tf.float32)))
            
        ds = ds.batch(batch_size)
       # ds = ds.repeat(repeat_count)
        ds = ds.prefetch(buffer_size=AUTOTUNE)
        return ds

    def hr_dataset(self):
        if not os.path.exists(self._hr_images_dir()):
            download_archive(self._hr_images_archive(), self.images_dir, extract=True)

        ds = self._images_dataset(self._hr_image_files()).cache(self._hr_cache_file())

        # creating a cache.
        # if not os.path.exists(self._hr_cache_index()):
        #    self._populate_cache(ds, self._hr_cache_file())

        return ds

    def lr_dataset(self):
        if not os.path.exists(self._lr_images_dir()):
            download_archive(self._lr_images_archive(), self.images_dir, extract=True)

        ds = self._images_dataset(self._lr_image_files()).cache(self._lr_cache_file())

        # creating a cache.
        # if not os.path.exists(self._lr_cache_index()):
        #    self._populate_cache(ds, self._lr_cache_file())

        return ds

    def _hr_cache_file(self):
        return os.path.join(self.caches_dir, f'DIV2K_{self.subset}_HR.cache')

    def _lr_cache_file(self):
        return os.path.join(self.caches_dir, f'DIV2K_{self.subset}_LR_{self.downgrade}_X{self.scale}.cache')

    def _hr_cache_index(self):
        return f'{self._hr_cache_file()}.index'

    def _lr_cache_index(self):
        return f'{self._lr_cache_file()}.index'

    def _hr_image_files(self):
        images_dir = self._hr_images_dir()
        return [os.path.join(images_dir, f'{image_id:04}.png') for image_id in self.image_ids]

    def _lr_image_files(self):
        images_dir = self._lr_images_dir()
        return [os.path.join(images_dir, self._lr_image_file(image_id)) for image_id in self.image_ids]

    def _lr_image_file(self, image_id):
        if not self._ntire_2018 or self.scale == 8:
            return f'{image_id:04}x{self.scale}.png'
        else:
            return f'{image_id:04}x{self.scale}{self.downgrade[0]}.png'

    def _hr_images_dir(self):
        return os.path.join(self.images_dir, f'DIV2K_{self.subset}_HR')

    def _lr_images_dir(self):
        if self._ntire_2018:
            return os.path.join(self.images_dir, f'DIV2K_{self.subset}_LR_{self.downgrade}')
        else:
            return os.path.join(self.images_dir, f'DIV2K_{self.subset}_LR_{self.downgrade}', f'X{self.scale}')

    def _hr_images_archive(self):
        return f'DIV2K_{self.subset}_HR.zip'

    def _lr_images_archive(self):
        if self._ntire_2018:
            return f'DIV2K_{self.subset}_LR_{self.downgrade}.zip'
        else:
            return f'DIV2K_{self.subset}_LR_{self.downgrade}_X{self.scale}.zip'

    @staticmethod
    def _images_dataset(image_files):
        ds = tf.data.Dataset.from_tensor_slices(image_files)
        ds = ds.map(tf.io.read_file)
        ds = ds.map(lambda x: tf.image.decode_png(x, channels=3), num_parallel_calls=AUTOTUNE)
     #   ds = ds.map(lambda x: tf.image.convert_image_dtype(x, dtype=tf.float32))
        return ds

    @staticmethod
    def _populate_cache(ds, cache_file):
        print(f'Caching decoded images in {cache_file} ...')
        for _ in ds: pass
        print(f'Cached decoded images in {cache_file}.')


def download_archive(file, target_dir, extract=True):
    source_url = f'http://data.vision.ee.ethz.ch/cvl/DIV2K/{file}'
    target_dir = os.path.abspath(target_dir)
    tf.keras.utils.get_file(file, source_url, cache_subdir=target_dir, extract=extract)
    os.remove(os.path.join(target_dir, file))


In [ ]:
model = get_super_resolution_model(1, 1, 64, 4)

In [ ]:
train_loader = SRDataLoader1(
            scale=4,            
            downgrade="bicubic",
            subset='train'
        )      
                         
train_ds = train_loader.dataset(
            batch_size=10,        
            random_transform=True, 
        )  


In [ ]:
train_ds

In [ ]:
len(train_ds)

In [ ]:
for i in train_ds:
    print(i[0])
    break

In [ ]:
val_loader = SRDataLoader1(
            scale=4,            
            downgrade="bicubic",
            subset='valid'
        )      
                         
val_ds = val_loader.dataset(
            batch_size=1,        # bcoz no transformation is applied, so samples have a different img_size.
            random_transform=False, 
        )  

In [ ]:
from utils import psnr_denoising

early_stopping_callback = keras.callbacks.EarlyStopping(
            monitor="val_psnr_denoising",
            patience=10,
            mode='max'
        )

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
            "dummy_training" + "best_model.h5",
            save_weights_only=True,
            monitor="val_psnr_denoising",
            mode="max",
            save_best_only=True,
            period=1
        )

reduce_lr_loss = keras.callbacks.ReduceLROnPlateau(
            monitor='val_psnr_denoising',
            factor=0.5,
            patience=5,
            verbose=1,
            epsilon=1e-7,
            mode='max'
        )
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
                optimizer=optimizer,
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=[psnr_denoising]
            )

model.fit(
                train_ds,
                validation_data=val_ds,
                epochs=5,
                callbacks=[early_stopping_callback, model_checkpoint_callback, reduce_lr_loss]
            )


In [ ]:
model.load_weights("dummy_trainingbest_model.h5")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("best_model.tflite", 'wb') as f:
    f.write(tflite_model)

In [ ]:
def inferrer(image, input_dims=(1, 400, 600, 3)):
    interpreter = tf.lite.Interpreter(model_path="best_model.tflite")
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.resize_tensor_input(input_details[0]['index'], input_dims)

    input_index = input_details[0]["index"]
    output_index = output_details[0]["index"]

    interpreter.allocate_tensors()
    interpreter.set_tensor(input_index, image)

    interpreter.invoke()
    output = interpreter.get_tensor(output_index)
    
    # Convert output array to image
    # output_image = (np.squeeze(output, axis=0).clip(0, 1) * 255).astype(np.uint8)

    # img = Image.fromarray(output_image)
    
    return output[0]

In [ ]:
import glob, time

lowlight_test_images_path = "test/super_resolution/"
test_files = glob.glob(lowlight_test_images_path + "*.png")

for test_file in tqdm.tqdm(test_files, total=len(test_files)):
    
        filename = test_file.split("/")[-1]
        lr_img = cv2.imread(test_file)
        lr_img = cv2.cvtColor(lr_img, cv2.COLOR_BGR2RGB)
        print(lr_img.shape)
        
        # for resizing specific model data to specific dim.
        lr_img = get_lowres_image(lr_img, mode="super_resolution")
        print(lr_img.shape)
        
        inputs = img_to_array(lr_img)
        inputs = np.expand_dims(inputs, axis=0)
        t = time.time()
        
        lr_img_shape = lr_img.shape
        inferrer_input_dims = (1, lr_img_shape[0], lr_img_shape[1], lr_img_shape[2])
        enhanced_image = inferrer(inputs, inferrer_input_dims)
        print("Time taken for inference: ", time.time() - t)

        if False:
            plt.figure()
            plt.subplot(131)
            plt.imshow(original_img)
            
            plt.subplot(132)
            plt.imshow(enhanced_image)
            
            plt.show()
        
        
        save_file_dir = lowlight_test_images_path.replace('test', 'results')
        save_file_path = save_file_dir  + filename
        print(save_file_path, "save path")
        cv2.imwrite(save_file_path, cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB))

In [ ]:
img_paths = glob.glob("results/super_resolution/*.png")
for img_path in img_paths:
#    n_rows = len(img_paths)
    img = cv2.imread(img_path)
    
    print(img.shape, "result shape")

    plt.figure()
    plt.imshow(img)
    plt.show()

In [ ]:
!git clone https://github.com/tuvovan/MIRNet-Keras.git

In [ ]:
import cv2 

img = cv2.imread('MIRNet-Keras/test/super/Canon_005_LR3.png')

In [ ]:
img = cv2.resize(img, (400, 260))

In [ ]:
img.shape

In [ ]:
plt.imshow(model(np.expand_dims(img, axis=0))[0]/255.0)